In [3]:
from datetime import datetime, timedelta
from persiantools.jdatetime import JalaliDate, JalaliDateTime
import pandas as pd
import plotly.express as px
import pandas as pd
import json


In [5]:
json_dir = 'temp-json/'
f = open(json_dir+'request.json', encoding='utf-8')
data = json.load(f)
f.close()
foods = data['foods']
foodTypes = data['foodTypes']
factors = data['factors']
factorItems = data['factorItems']
restaurant = data['restaurant']
# foods # 'id', 'name', 'real_name', 'name_arabic', 'name_latin', 'discount', 'price', 'scale', 'cooking_duration', 'restaurant_id', 'food_type_id', 'cashback', 'real_price', 'packing_price', 'maximum_daily_capacity', 'maximum_hourly_capacity', 'created_at', 'updated_at', 'deleted_at',
# foodTypes # 'id', 'name', 'real_name', 'name_latin', 'discount', 'cooking_duration', 'restaurant_id',  'created_at', 'updated_at', 'deleted_at',
# factors # 'id','price','phone_number','description','date','time','day_name','restaurant_id','created_at','updated_at','deleted_at',Items # 'id', 'food_id', 'factor_id', 'name', 'amount', 'scale', 'price', 'price_total', 'restaurant_id', 'created_at', 'updated_at', 'deleted_at'
# factorItems # 'id','food_id':,'factor_id','name','amount','scale','price','price_total','restaurant_id','created_at','updated_at','deleted_at',
# restaurant # 'id','owner_id','name','description','location','created_at','updated_at','deleted_at'
factors = {d['id']: d for d in factors}


#df = px.data.gapminder().query("continent=='Oceania'")
#fig = px.line(df, x="year", y="lifeExp", color='country', symbol="country", markers=True, text="year")
#fig.show()

foodGraphDaily = []
foodGraphWeekly = []
foodGraphMonthly = []

food_ids = [1]
foodType_ids = [1]
fromDate = JalaliDate(1400, 8, 1)  # from < to
toDate = JalaliDate(1401, 4, 1) #JalaliDate(datetime.today())    JalaliDate(datetime.today() - timedelta(days=dayShift))
dayCounter = -1
lastJdate = JalaliDate(datetime.today())
itemBuffer = []
dataBuffer = {}

for idx, item in enumerate(factorItems):
    date1 = factors[int(item['factor_id'])]['date']
    time1 = factors[int(item['factor_id'])]['time']
    jdate = JalaliDateTime(int(date1[0:4]),int(date1[5:7]),int(date1[8:10]), int(time1[0:2]),int(time1[3:5]),int(time1[6:8]))
    #print(jdate.date())
    if jdate.date() <= fromDate:
        continue
    if jdate.date() > toDate:
        break
    # setting the vars for the first time in the loop
    if dayCounter == -1:
        lastJdate = jdate
        dayCounter = 0
    # collecting every factors of the date
    if lastJdate.date() == jdate.date():
        dataBuffer = item
        dataBuffer['date'] = date1
        dataBuffer['time'] = time1
        itemBuffer.append(item)
        lastJdate = JalaliDateTime(int(date1[0:4]),int(date1[5:7]),int(date1[8:10]), int(time1[0:2]),int(time1[3:5]),int(time1[6:8]))
    # processing the factors which are collected of one day
    # itemBuffer contains all factors of the day
    else: 
        dayCounter = dayCounter + 1
        dataBuffer = {
            'date':factors[int(itemBuffer[0]['factor_id'])]['date'],
            'sumPayment':sum( [element['price_total'] for idx1, element in  enumerate(itemBuffer)] ),
            'factorNumber':len(itemBuffer),
            'dayName':factors[int(itemBuffer[0]['factor_id'])]['date'],
        }
        dataBuffer['08-18 (count)'] = len( [element["amount"] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('08:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('17:59:59','%H:%M:%S') ] )
        dataBuffer['18-20 (count)'] = len( [element["amount"] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('18:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('19:59:59','%H:%M:%S') ] )
        dataBuffer['20-21 (count)'] = len( [element["amount"] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('20:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('20:59:59','%H:%M:%S') ] )
        dataBuffer['21-22 (count)'] = len( [element["amount"] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('21:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('21:59:59','%H:%M:%S') ] )
        dataBuffer['22-23 (count)'] = len( [element["amount"] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('22:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('22:59:59','%H:%M:%S') ] )
        dataBuffer['23-24 (count)'] = len( [element["amount"] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('23:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('23:59:59','%H:%M:%S') ] )
        dataBuffer['24-08 (count)'] = len( [element["amount"] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('00:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('07:59:59','%H:%M:%S') ] )
        dataBuffer['08-18 (payment)'] = sum( [element['price_total'] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('08:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('17:59:59','%H:%M:%S') ] )
        dataBuffer['18-20 (payment)'] = sum( [element['price_total'] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('18:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('19:59:59','%H:%M:%S') ] )
        dataBuffer['20-21 (payment)'] = sum( [element['price_total'] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('20:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('20:59:59','%H:%M:%S') ] )
        dataBuffer['21-22 (payment)'] = sum( [element['price_total'] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('21:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('21:59:59','%H:%M:%S') ] )
        dataBuffer['22-23 (payment)'] = sum( [element['price_total'] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('22:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('22:59:59','%H:%M:%S') ] )
        dataBuffer['23-24 (payment)'] = sum( [element['price_total'] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('23:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('23:59:59','%H:%M:%S') ] )
        dataBuffer['24-08 (payment)'] = sum( [element['price_total'] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('00:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('07:59:59','%H:%M:%S') ] )
        dataBuffer['24-08 (payment)'] = sum( [element['price_total'] for idx1, element in  enumerate(itemBuffer) if datetime.strptime(element["time"],'%H:%M:%S') > datetime.strptime('00:00:00','%H:%M:%S') and datetime.strptime(element["time"],'%H:%M:%S') < datetime.strptime('07:59:59','%H:%M:%S') ] )
        foodGraphDaily.append(dataBuffer)
        itemBuffer = []
        #
        dataBuffer = item
        dataBuffer['date'] = date1
        dataBuffer['time'] = time1
        itemBuffer.append(item)
        lastJdate = JalaliDateTime(int(date1[0:4]),int(date1[5:7]),int(date1[8:10]), int(time1[0:2]),int(time1[3:5]),int(time1[6:8]))



In [6]:
food_df = pd.DataFrame(data=foodGraphDaily)
#fig = px.line(food_df, x='date', y='21-22 (payment)', color='21-22 (payment)', symbol='21-22 (payment)', markers=True, text='21-22 (payment)')
fig = px.line(food_df, x='date', y='21-22 (payment)',  markers=True, text='21-22 (payment)')
fig.show()
fig2 = px.line(food_df, x='date', y='sumPayment', markers=True, text='sumPayment')
fig2.show()
    #
    #if item['food_id'] in food_ids:
    #    foodGraph[int(item['food_id'])] = 1
    #a=1
    #hourly
    #if checkConditionOfFactor(factors[item['factor_id']], day=0, week=0, month=0):
    #daily

def checkConditionOfFactor(condition):
    if condition == 's':
        return true
#    return true


In [8]:
food_df.head()

,date,sumPayment,factorNumber,dayName,08-18 (count),18-20 (count),20-21 (count),21-22 (count),22-23 (count),23-24 (count),24-08 (count),08-18 (payment),18-20 (payment),20-21 (payment),21-22 (payment),22-23 (payment),23-24 (payment),24-08 (payment)
0,1400-08-02,33415.0,69,1400-08-02,4,16,10,16,9,14,0,1250.0,5885.0,6075.0,7980.0,4105.0,8120.0,0.0
1,1400-08-04,36590.0,61,1400-08-04,0,16,13,17,14,1,0,0.0,7420.0,8465.0,12285.0,6950.0,1470.0,0.0
2,1400-08-03,18935.0,38,1400-08-03,0,2,11,11,9,5,0,0.0,1760.0,3895.0,5380.0,5230.0,2670.0,0.0
3,1400-08-05,22230.0,42,1400-08-05,0,0,19,12,5,6,0,0.0,0.0,9960.0,5750.0,2320.0,4200.0,0.0
4,1400-08-06,39345.0,68,1400-08-06,2,7,20,15,16,8,0,480.0,2730.0,11240.0,11080.0,7205.0,6610.0,0.0


In [11]:

fig = px.line(food_df, x='date', y='18-20 (count)',  markers=True, text='21-22 (payment)')
fig.show()
fig2 = px.line(food_df, x='date', y='factorNumber', markers=True, text='sumPayment')
fig2.show()



In [23]:
fig = px.bar(food_df, x='date', y='21-22 (payment)',  text='21-22 (payment)')
fig.show()
fig2 = px.bar(food_df, x='08-18 (count)', y='factorNumber', text='factorNumber')
fig2.show()


In [22]:
fig = px.pie(food_df, names='08-18 (count)', values='factorNumber', title='sumPayment')
fig.show()


fig = px.pie(food_df, names='18-20 (count)', values='factorNumber', title='sumPayment')
fig.show()

In [26]:
fig = px.pie(food_df, names='08-18 (count)', values='factorNumber',hole=0.6 )
fig.show()


fig = px.pie(food_df, names='18-20 (count)', values='factorNumber', hole=0.6)
fig.show()